In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import cv2
import numpy as np
import random
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [2]:
!git clone https://github.com/haianh23102000/hoc_may.git

Cloning into 'hoc_may'...
remote: Enumerating objects: 8344, done.
remote: Counting objects: 100% (8344/8344), done.
remote: Compressing objects: 100% (8338/8338), done.
remote: Total 8344 (delta 7), reused 8327 (delta 2), pack-reused 0
Receiving objects: 100% (8344/8344), 488.02 MiB | 33.58 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (8327/8327), done.


In [3]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes
def prepare_data(size):
  transform_train = transforms.Compose([
    transforms.Resize((size,size)),
    transforms.ToTensor()  ])
  transform_test = transforms.Compose([
    transforms.Resize((size,size)), 
    transforms.ToTensor()  ])
  trainset = torchvision.datasets.ImageFolder(root='/content/hoc_may/Data/train', transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root='/content/hoc_may/Data/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

In [4]:
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

In [5]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true = []
  pred = []
  model.train()
  running_loss = 0.0
  reporting_steps = 18
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred += list(predicted.cpu().numpy())
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return pred, true

In [6]:
def test_epoch(epoch, model, loader, device):
  true = []
  pred = [] 
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      true += list(labels.cpu().numpy())
      pred += list(predicted.cpu().numpy())
  return pred, true

In [7]:
def main(model = 'vgg16', size = 32):
  classes = get_classes()
  datasets = prepare_data(size)
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  if model == 'vgg16':
    print("vgg16")
    model = torchvision.models.vgg16()
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    print("vgg19")
    model = torchvision.models.vgg19()
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    print("resnet50")
    model = torchvision.models.resnet50()
    model.fc = torch.nn.Linear(in_features=2048, out_features=3)
  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))

In [10]:
main('vgg16',32)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


vgg16
Epoch 0 step 17 ave_loss 1.0981
Epoch 0 step 35 ave_loss 1.0978
Epoch 0 step 53 ave_loss 1.0982
Epoch 0 step 71 ave_loss 1.0973
Epoch 0 step 89 ave_loss 1.0902
Epoch 0 step 107 ave_loss 1.0719
Epoch 0 step 125 ave_loss 1.0533
Epoch 0 step 143 ave_loss 1.0974
Epoch 0 step 161 ave_loss 1.0972
Epoch 0 step 179 ave_loss 1.0759
Epoch 0 step 197 ave_loss 0.9690
              precision    recall  f1-score   support

          2C       0.37      1.00      0.54       409
          3C       0.03      0.01      0.01       367
          4C       0.98      0.47      0.64       831

    accuracy                           0.50      1607
   macro avg       0.46      0.49      0.40      1607
weighted avg       0.61      0.50      0.47      1607

Epoch 1 step 17 ave_loss 0.7917
Epoch 1 step 35 ave_loss 0.6066
Epoch 1 step 53 ave_loss 0.5436
Epoch 1 step 71 ave_loss 0.5579
Epoch 1 step 89 ave_loss 0.4596
Epoch 1 step 107 ave_loss 0.3268
Epoch 1 step 125 ave_loss 0.4395
Epoch 1 step 143 ave_loss 0.2